a)make facebook social network graph

In [1]:
import networkx as nx
import pandas as pd

# Load the data from the file
df = pd.read_csv('facebook_combined.txt', sep=" ", header=None)
df.columns = ["origin", "destination"]

# Create the graph from the dataframe
G = nx.from_pandas_edgelist(df, 'origin', 'destination')

# Get the number of nodes and edges in the graph
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print(f"The graph has {num_nodes} nodes and {num_edges} edges.")


The graph has 4039 nodes and 88234 edges.


b) make 10 realization from graph

In [2]:
import numpy as np

np.random.seed(0)

realizations = []
for _ in range(10):

    G_copy = G.copy()


    for edge in G_copy.edges():

        rand_num = np.random.rand()


        if rand_num > 0.1:
            G_copy.remove_edge(*edge)

    # Add the realization to the list
    realizations.append(G_copy)



In [3]:
# Print the number of edges and nodes for each realization
for i, realization in enumerate(realizations):
    num_nodes = realization.number_of_nodes()
    num_edges = realization.number_of_edges()
    print(f"Realization {i+1}: {num_nodes} nodes, {num_edges} edges")

Realization 1: 4039 nodes, 8933 edges
Realization 2: 4039 nodes, 8803 edges
Realization 3: 4039 nodes, 8920 edges
Realization 4: 4039 nodes, 8993 edges
Realization 5: 4039 nodes, 8878 edges
Realization 6: 4039 nodes, 8655 edges
Realization 7: 4039 nodes, 8934 edges
Realization 8: 4039 nodes, 8747 edges
Realization 9: 4039 nodes, 8650 edges
Realization 10: 4039 nodes, 8842 edges


c) calculate f(s) for top 8 node and for 8 random node

In [4]:
# Get the degrees of all nodes in the graph
degrees = G.degree()

# Sort the nodes by degree in descending order
sorted_nodes = sorted(degrees, key=lambda x: x[1], reverse=True)

# Select the top 8 nodes
top_nodes = [node for node, degree in sorted_nodes[:8]]

print(f"The top 8 nodes with the highest degree are: {top_nodes}")


The top 8 nodes with the highest degree are: [107, 1684, 1912, 3437, 0, 2543, 2347, 1888]


In [6]:
import random

# Get all nodes in the graph
all_nodes = list(G.nodes())

random.seed(42)
# Randomly select 8 nodes
random_nodes = random.sample(all_nodes, 8)

print(f"8 randomly selected nodes are: {random_nodes}")


8 randomly selected nodes are: [3078, 970, 102, 2049, 1640, 1517, 1428, 1085]


In [7]:
def influence_spread(S, realizations):
    total = 0
    for G in realizations:
        visited = {node: False for node in G.nodes()}
        for node in S:
            if not visited[node]:
                for reachable_node in nx.bfs_tree(G, node):
                    visited[reachable_node] = True
        total += sum(visited.values())
    return total / len(realizations)


In [8]:
f_value_top_nodes = influence_spread(top_nodes, realizations)
f_value_random_nodes = influence_spread(random_nodes, realizations)

print(f"The value of f(S) for the top nodes is: {f_value_top_nodes}")
print(f"The value of f(S) for the random nodes is: {f_value_random_nodes}")


The value of f(S) for the top nodes is: 2956.3
The value of f(S) for the random nodes is: 2943.0


d) celf

In [61]:
import heapq
from tqdm import tqdm

# Lazy hill climbing within CELF algorithm for benefit-cost greedy approach
def celf_unit_cost(realizations):
  count = 0
  S_prime = []
  marginal_gains = []
  influence_spread_sorted = []
  # Initialize the min heap
  min_heap = []
  for k in tqdm(range(8), desc="Building solution(unit cost)"):
    if k==0 :
      # Calculate the marginal gain for each node
      for node in G.nodes():

        if node not in S_prime:
            S_prime.append(node)
            gain = influence_spread(S_prime, realizations)
            S_prime.remove(node)
            influence_spread_sorted.append(( node , gain ))

      influence_spread_sorted.sort(key=lambda x: x[1] ,reverse=True)

      marginal_gains.append(influence_spread_sorted[0][1])

      S_prime.append(influence_spread_sorted[0][0])

      del influence_spread_sorted[0]

      spread = influence_spread_sorted[0][1]

    else:

      while True :

        top_currnet_node = influence_spread_sorted[0][0]

        #calculate and replacce influence spread for new node with updated selected nodes

        S_prime.append(top_currnet_node)
        gain = influence_spread(S_prime, realizations) - spread
        S_prime.remove(top_currnet_node)

        #replace vlaue of influnce spread in the sorted list

        influence_spread_sorted[0] = (top_currnet_node , gain)

        influence_spread_sorted.sort(key=lambda x: x[1] ,reverse=True)

        if top_currnet_node == influence_spread_sorted[0][0] :

          break

      S_prime.append(influence_spread_sorted[0][0])

      spread += influence_spread_sorted[0][0]

      del influence_spread_sorted[0]

  return S_prime


In [64]:
import time

start_time = time.time()

S_prime = celf_unit_cost(realizations)

end_time = time.time()

execution_time_of_celf_unitcost = end_time - start_time

Building solution(unit cost): 100%|██████████| 8/8 [2:21:04<00:00, 1058.01s/it]


In [65]:
print(f"The value of f(S) for the celf is:{influence_spread(S_prime, realizations)}" )
print(f" S_prime : { S_prime } ")

The value of f(S) for the celf is:3074.1
 S_prime : [107, 803, 187, 3980, 4017, 3538, 3008, 3155] 
